In [ ]:
from glob import glob

In [ ]:
edits_tsv_list = glob("/mnt/spellcheck_data/data/c4dataset/edits_tsv/*")

In [ ]:
with open(edits_tsv_list[0]) as f:
    current_edit_md5 = None
    for edits_tsv_line in f:
        try:
            edit_md5, byte_start, byte_end, text = edits_tsv_line.strip("\n").split(
                "\t", 3)
        except ValueError:
            pass  # Skip malformed lines
        else:
            if edit_md5 != current_edit_md5:
                if current_edit_md5 is not None:
                    print("IN",current_edit_md5, current_edits)
                current_edit_md5 = edit_md5
                current_edits = []
            current_edits.append((int(byte_start), int(byte_end), text))
            print(current_edits)
    # edits_iterator = get_edits(f)
    # print(next(edits_iterator))

In [ ]:
def get_edits(edits_tsv_reader):
  """Generator method for the edits."""
  current_edit_md5 = None
  for edits_tsv_line in edits_tsv_reader:
    try:
      edit_md5, byte_start, byte_end, text = edits_tsv_line.strip("\n").split(
          "\t", 3)
    except ValueError:
      pass  # Skip malformed lines
    else:
      if edit_md5 != current_edit_md5:
        if current_edit_md5 is not None:
          yield current_edit_md5, current_edits
        current_edit_md5 = edit_md5
        current_edits = []
      current_edits.append((int(byte_start), int(byte_end), text))
  yield current_edit_md5, current_edits


In [ ]:
data = {}
with open("/home/parth.chudasama/data_mnt/target_data/target_sentencs_00000-of-00010") as f:
    for target_sentence_tsv_line in f:
          md5, target_text = target_sentence_tsv_line.strip("\n").split("\t", 1)
          data.update({md5:target_text})


In [ ]:
import errant

annotator = errant.load('en')

In [ ]:
orig = annotator.parse('This are gramamtical sentence.', tokenise=True)
cor = annotator.parse('This is a grammatical sentence.',tokenise=True)
edits = annotator.annotate(orig, cor)
for e in edits:
    print(e.o_start, e.o_end, e.o_str, e.c_start, e.c_end, e.c_str, e.type)

In [ ]:
import os
os.path.basename("/home/parth.chudasama/data_mnt/target_pairs/target_pairs_00000-of-00010")

In [ ]:
data = []
count = 0
with open("/home/parth.chudasama/data_mnt/target_pairs/target_pairs_00000-of-00010") as f:
    for line in f:
        src, trg = line.strip().split("\t")
        data.append((src, trg))
        count+=1
        if count>2000:
            break

In [ ]:
len(data)

In [ ]:
import boto3
import uuid
import json
sqsResource = boto3.resource('sqs')

def write_sqs(integerList):
    queue = sqsResource.get_queue_by_name(QueueName="C4_data_process")
    entries = []

    for src, trg in integerList:
        entry =  {
            'Id': str(uuid.uuid4()),
            'MessageBody': json.dumps(
                {"src": src,
                 "trg": trg}
                )
        }

        entries.append(entry)
    # return  entries
    response = queue.send_messages(Entries=entries)


In [ ]:
from tqdm.notebook import tqdm

In [ ]:

with open("/home/parth.chudasama/data_mnt/target_pairs/target_pairs_00000-of-00010") as f:
    for line in tqdm(f):
        src, trg = line.strip().split("\t")
        data.append((src, trg))
        if len(data)==10:
            write_sqs(data)
            data = []

In [ ]:
import dask.dataframe as dd
df = dd.read_csv('~/data_mnt/target_pairs/target_pairs_00000-of-00010', sep='\t', names=['src','trg'])

In [ ]:
df.head()

In [21]:
import dask.dataframe as dd

In [24]:
df = dd.read_json('c4_json_data/*/*',lines=True)
df.head()

,src,trg,edits,file
0,"015 Buck , marilyn J.","015 Buck , Marilyn J.","[{'o_start': 3, 'o_end': 4, 'o_str': 'marilyn'...",0
1,Mark Mohler said in a post on Instagram that h...,"On Wednesday, diver Mark Mohler said in a post...","[{'o_start': 0, 'o_end': 0, 'o_str': '', 'e_st...",0
2,"Momover, these devices have been proven to hel...","Moreover, these devices are proven to help con...","[{'o_start': 0, 'o_end': 1, 'o_str': 'Momover,...",0
3,Chris is a chartered accountant who has held a...,Chris is a chartered accountant who was Chief ...,"[{'o_start': 6, 'o_end': 7, 'o_str': 'has', 'e...",0
4,Research report of how local authorities have ...,Research report on how local authorities have ...,"[{'o_start': 2, 'o_end': 3, 'o_str': 'of', 'e_...",0


In [26]:
df['e_flag'] = df.map_partitions(check_list, meta=(None,'bool'))

In [27]:
df.loc[df['e_flag']]

,src,trg,edits,file,e_flag
npartitions=404,,,,,
,object,object,object,int64,bool
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [ ]:
df.to_csv("c4_only_spell_filtered-*.csv",index=False)

In [2]:
from glob import glob
import pandas as pd
from tqdm import tqdm
from mpire import WorkerPool
import os
import uuid

In [5]:
file_list = glob("c4_json_data/*/*")

In [20]:
def check_list(_list):
    for i in _list:
        if i['tag'] in ['R:ORTH', 'R:SPELL']:
            return True
    return False

def check_edits(edit_list):
    for i in _list:
        if i['tag'] in ['R:ORTH', 'R:SPELL']:
            
    

def filter_df(file_path):
    count= str(uuid.uuid4())
    df = pd.read_json(file_path, lines=True)
    df['e_flag'] = df['edits'].apply(lambda x: check_list(x))
    df = df.loc[df['e_flag']]
    df.to_csv(f"c4_only_spell_filtered/c4_only_spell_filtered-{count}.csv")

In [21]:
with WorkerPool(n_jobs=os.cpu_count() - 1) as pool:
    sentence_data = pool.map(filter_df, file_list, progress_bar=True)

  0%|          | 0/404 [00:00<?, ?it/s]

In [3]:
file_list_spell = glob("c4_only_spell_filtered/sentences_with_spell/*")

In [5]:
df = pd.read_csv(file_list_spell[0])

In [6]:
df

,Unnamed: 0,src,trg,edits,file,e_flag
0,1,We are available in every DTH platforms in reg...,We are available on all DTH platforms in the r...,"[{'o_start': 3, 'o_end': 4, 'o_str': 'in', 'e_...",4,True
1,7,Details: FRRD SHIPPING on orders of 150 $Expir...,Details: FREE SHIPPING on orders of $150. Expi...,"[{'o_start': 1, 'o_end': 2, 'o_str': 'FRRD', '...",4,True
2,9,I came back to coming from the Party extremly ...,I came back from the party extremely excited b...,"[{'o_start': 3, 'o_end': 5, 'o_str': 'to comin...",3,True
3,13,"As the President reminded us, what better demo...","As the President reminded us, what better demo...","[{'o_start': 13, 'o_end': 14, 'o_str': 'iracy'...",3,True
4,14,November.1st's gleasons and mendez gym have be...,November 1st Gleasons and Mendez Gym become dr...,"[{'o_start': 0, 'o_end': 1, 'o_str': ""November...",4,True
...,...,...,...,...,...,...
46526,134495,"Gruden will call the offensive plays, not Olso...","Gruden will call the offensive plays, not Olso...","[{'o_start': 11, 'o_end': 12, 'o_str': 'then',...",4,True
46527,134502,Video - do you consider a chaplain?,Video – Are You Considering Becoming a Chaplain?,"[{'o_start': 1, 'o_end': 2, 'o_str': '-', 'e_s...",4,True
46528,134503,The Hocking Valley Scenic Railway Depot has it...,The Hocking Valley Scenic Railway depot featur...,"[{'o_start': 5, 'o_end': 6, 'o_str': 'Depot', ...",4,True
46529,134505,"Terri Schwartz, meanwhile praised the performa...","Terri Schwartz, meanwhile, praised the perform...","[{'o_start': 2, 'o_end': 3, 'o_str': 'meanwhil...",4,True
